# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
brent = pd.read_csv('data/Brent Oil Futures Historical Data.csv', thousands=',')
brent.Date = pd.to_datetime(brent.Date)
brent = brent.set_index('Date')['Price'].astype('float')

In [3]:
crude = pd.read_csv('data/Crude Oil WTI Futures Historical Data.csv', thousands=',')
crude.Date = pd.to_datetime(crude.Date)
crude = crude.set_index('Date')['Price'].astype('float')

In [4]:
gasoline = pd.read_csv('data/Gasoline RBOB Futures Historical Data.csv', thousands=',')
gasoline.Date = pd.to_datetime(gasoline.Date)
gasoline = gasoline.set_index('Date')['Price'].astype('float')

In [5]:
heating = pd.read_csv('data/Heating Oil Futures Historical Data.csv', thousands=',')
heating.Date = pd.to_datetime(heating.Date)
heating = heating.set_index('Date')['Price'].astype('float')

In [6]:
gas = pd.read_csv('data/London Gas Oil Futures Historical Data.csv', thousands=',')
gas.Date = pd.to_datetime(gas.Date)
gas = gas.set_index('Date')['Price'].astype('float')

In [7]:
natural = pd.read_csv('data/Natural Gas Futures Historical Data.csv')
natural.Date = pd.to_datetime(natural.Date)
natural = natural.set_index('Date')['Price'].astype('float')

Fill nan by previous prices

In [8]:
prices = pd.DataFrame(
    {'brent': brent, 'crude': crude, 'gasoline': gasoline, 'heating': heating, 'gas': gas, 'natural': natural}
)
prices.index.name = 'date'
prices = prices.fillna(method='ffill')
prices.head()

,brent,crude,gasoline,heating,gas,natural
date,,,,,,
2008-01-02,97.84,99.62,2.4818,2.7404,855.00,7.850
2008-01-03,97.60,99.18,2.4793,2.7191,860.50,7.674
2008-01-04,96.79,97.91,2.4677,2.6835,846.50,7.841
2008-01-07,94.39,95.09,2.3965,2.5935,816.50,7.879
2008-01-08,95.54,96.33,2.4263,2.6363,839.25,7.967


In [9]:
prices_next = prices.loc[:]
for col in prices.columns:
    prices_next['{}_next'.format(col)] = prices[col][1:].tolist() + [None]
prices_next = prices_next.iloc[:-1]
prices_next.head()

,brent,crude,gasoline,heating,gas,natural,brent_next,crude_next,gasoline_next,heating_next,gas_next,natural_next
date,,,,,,,,,,,,
2008-01-02,97.84,99.62,2.4818,2.7404,855.00,7.850,97.60,99.18,2.4793,2.7191,860.50,7.674
2008-01-03,97.60,99.18,2.4793,2.7191,860.50,7.674,96.79,97.91,2.4677,2.6835,846.50,7.841
2008-01-04,96.79,97.91,2.4677,2.6835,846.50,7.841,94.39,95.09,2.3965,2.5935,816.50,7.879
2008-01-07,94.39,95.09,2.3965,2.5935,816.50,7.879,95.54,96.33,2.4263,2.6363,839.25,7.967
2008-01-08,95.54,96.33,2.4263,2.6363,839.25,7.967,94.37,95.67,2.3884,2.6134,844.25,8.099


In [10]:
returns = pd.DataFrame(index=prices_next.index)
for col in prices.columns:
    returns[col] = (prices_next['{}_next'.format(col)] - prices_next[col]) / prices_next[col]
returns.head()

,brent,crude,gasoline,heating,gas,natural
date,,,,,,
2008-01-02,-0.002453,-0.004417,-0.001007,-0.007773,0.006433,-0.022420
2008-01-03,-0.008299,-0.012805,-0.004679,-0.013093,-0.016270,0.021762
2008-01-04,-0.024796,-0.028802,-0.028853,-0.033538,-0.035440,0.004846
2008-01-07,0.012183,0.013040,0.012435,0.016503,0.027863,0.011169
2008-01-08,-0.012246,-0.006851,-0.015620,-0.008686,0.005958,0.016568


In [11]:
returns.to_csv('data/returns.csv')
prices.to_csv('data/prices.csv')

In [12]:
returns_train = returns.iloc[:2432]
returns_test = returns.iloc[2433:]
returns_train.to_csv('data/returns_train.csv')
returns_test.to_csv('data/returns_test.csv')